In [ ]:
# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.image import imread
import scipy.io
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

import os
import markdown
import random
from scipy.integrate import quad

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import oct2py
from oct2py import octave
%load_ext oct2py.ipython

In [ ]:
%%capture
os.chdir(os.getcwd() + '/qMRLab')
%octave startup

In [ ]:
# Add fun folder
octave.addpath('../fun')

<center><h1 style="font-family: timesnewroman;font-size: 40px;">Quantitative Magnetization Transfer using Spoiled Gradient echo</h1></center>
<p>

<div class=blog_body>
<p style="text-align:justify;">
Magnetization Transfer (MT) has been extensively applied to study macromolecular biological tissue composition. The imaging contrast resides in the magnetization transfer between free-water protons and macromolecular proton compartments, through chemical exchange and dipolar interactions. In the two-pool tissue model, highly mobile protons are associated with the free-water pool while protons found in semisolid macromolecular sites are defined as the restricted pool (Sled 2018). A simple method for visualizing MT effects includes acquiring two images with and without an off-resonance MT pulse to calculate the MT ratio (MTR), which is the normalized difference of these two images (Wolff and Balaban 1989). Despite its proven usefulness to study multiple sclerosis (Zheng et al. 2018), Alzheimer’s disease (Fornari et al. 2012) and psychiatric disorders (Chen et al. 2015), the MTR is a semi-quantitative metric that depends critically on the imaging sequence parameters (Wood and Malik 2020). Another semi-quantitative approach is the estimation of MT saturation (MTsat) by fitting the MT signal obtained from an MT-weighted (MTw), a proton density (PD) weighted and T<sub>1</sub>-weighted (T<sub>1</sub>w) contrast (Helms et al. 2008). Quantitative MT (qMT) consists of fitting multiple images to a mathematical model to extract tissue-specific parameters related to physical quantities, such as pool sizes, magnetization exchange rates between pools, and T<sub>1</sub>,T<sub>2</sub> relaxation times of each pool. Compared to semi-quantitative approaches (MTR, MTsat), qMT has long acquisition protocols and sometimes needs additional measurements (eg. B<sub>0</sub>, B<sub>1</sub>, T<sub>1</sub>), and the complex models required to fit the quantitative maps makes it a challenging imaging technique.
</p>

<p style="text-align:justify;">
In 2015, our lab published qMTLab (Cabana et al. 2015), an open-source software project seeking to unify three qMT methods in the same interface: qMT using spoiled gradient echo (qMT-SPGR), qMT using balanced steady-state free precession (qMT-bSSFP), and qMT using selective inversion recovery with fast spin echo (qMT-SIRFSE). qMTLab allowed users to simulate, evaluate, fit, and visualize qMT data with the possibility to share qMT protocols between researchers, allowing them to compare the performance of their methods (Cabana et al. 2015). Since then, we have extended the project and renamed it to qMRLab, (Karakuzu et al. 2020), which in addition to qMT now provides over 20 quantitative techniques under one umbrella, such as relaxation and diffusion models, quantitative susceptibility mapping, B<sub>0</sub> and B<sub>1</sub> mapping. In addition, we created interactive tutorials (Boudreau 2018a; Boudreau 2018b; Boudreau 2019) and blog posts for several qMRI techniques that were published under a creative commons license, and one of these tutorials (Boudreau 2018a) made it into a quantitative MRI book published by Elsevier (Seiberlich et al. 2020). This blog post is a continuation of our qMRLab outreach initiative, where we will focus on qMT and the tools we provide in qMRLab for this class of  techniques.
</p>
    
<p style="text-align:justify;">
Below is an introduction to qMT (with a focus on qMT-SPGR), where we will cover signal modelling and data fitting.
</p>
</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Signal Modelling</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
The magnetization transfer in a two-pool model is modelled by a set of coupled differential equations (Sled and Pike 2000):
</p>

<p style="text-align:justify;">
<center><img src="equation1-5.png" style="width:auto;height:250px;margin-bottom: 30px;margin-top: 30px;"></center>
</p>

<p style="text-align:justify;">
where the magnetization at time $t$ is given by M = [M<sub>x,f</sub>, M<sub>y,f</sub>, M<sub>z,f</sub>, M<sub>z,r</sub>] and M<sub>x,f</sub> and M<sub>y,f</sub> are the transverse magnetization for the free pool in the $x$ and $y$ direction, respectively. The longitudinal magnetization for the free and restricted pool is denoted by M<sub>z,f</sub> and M<sub>z,r</sub>. Due to the very short T<sub>2</sub> of the restricted pool (on the order of microseconds), the transverse magnetization of this pool is not explicitly modelled.
</p>
    
<p style="text-align:justify;">
The constants k<sub>f</sub> and k<sub>r</sub> represent the exchange rate of the longitudinal magnetization from the free pool to the restricted pool (k<sub>f</sub>) and from the restricted to the free pool (k<sub>r</sub>). The ratio of these quantities is constrained by the size of the restricted to the free-water pool, expressed as k<sub>f</sub>/k<sub>r</sub> = M<sub>0,r</sub>/M<sub>0,f</sub>. This ratio is called the pool size ratio $F$ defined as F = M<sub>0,r</sub>/M<sub>0,f</sub>. The precessional frequency ω<sub>1</sub> is a measure of the power of the off-resonance radiofrequency pulse and ∆ is a frequency offset at which the magnetic B<sub>1</sub><sup>+</sup> field is applied. As shown in Figure 1, an on-resonance (∆ = 0) radiofrequency pulse is also part of the pulse sequence and is applied after the MT pulse. The relaxation time constants for the free and restricted pools are denoted by T<sub>1,f</sub> and T<sub>1,r</sub> for the longitudinal magnetization, and T<sub>2,f</sub> and T<sub>2,r</sub> for the transverse magnetization. Finally, $W$ is the saturation rate of the restricted pool, which is a function of the absorption lineshape $G$ that depends on the frequency offset and the transverse relaxation of the restricted pool (Figure 2).
</p>
</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 1. Simplified pulse sequence diagram of a magnetization transfer spoiled gradient (MT-SPGR) experiment with an MT pulse followed by a spoiler gradient to destroy any transverse magnetization before the application of the on-resonance excitation pulse.
</b>
</p>
</div>

<p>
<center><img src="mtspgr_pulsesequence.png" style="width:500px;height:auto;"></center>

<div class=blog_body>
<p style="text-align:justify;">
The absorption lineshape of the restricted pool that best characterizes the proton system depends on the environment. For simple systems such as agar gel phantoms, the Gaussian lineshape describes magnetization transfer effects well (Hankelman et al. 1993), while for more complicated and biologically relevant models, the super-Lorentzian lineshape is the best choice (Morrison et al. 1995). 
</p>

</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 2. Gaussian, Lorentzian and super-Lorentzian absorption lineshapes plotted as a function of the frequency offset ∆ and T<sub>2,r</sub>.
</b>
</p>
</div>

In [ ]:
def integrand(x, delta, t_2r):

    return (1/abs(3*x**2 - 1))*np.exp(-2*((2*np.pi*delta*t_2r)/(3*x**2 - 1))**2)

In [ ]:
init_notebook_mode(connected=True)

delta = np.arange(-8000,8100,100) # in Hz
t_2r = np.arange(6e-6,24e-6,2e-6) # in s
G_gaussian = np.zeros((len(delta),len(t_2r)+1))
G_lor = np.zeros((len(delta),len(t_2r)+1))
G_superlor = np.zeros((len(delta),len(t_2r)+1))

G_gaussian[:,0] = delta
G_lor[:,0] = delta
G_superlor[:,0] = delta

for ii in range(len(t_2r)):
    for jj in range(len(delta)):
        G_gaussian[jj,ii+1] = (t_2r[ii]/np.sqrt(2*np.pi))*np.exp(-(2*np.pi*delta[jj]*t_2r[ii])**2/2)
        
        G_lor[jj,ii+1] = (t_2r[ii]/np.pi)*1/(1+((2*np.pi*delta[jj]*t_2r[ii])**2))
        
        integral = quad(integrand, 0, 1, args=(delta[jj],t_2r[ii]))
        G_superlor[jj,ii+1] = (t_2r[ii])*(np.sqrt(2/np.pi))*integral[0]


init_notebook_mode(connected=True)

lineshape1 = [dict(
        visible = False,
        x = G_gaussian[:,0],
        y = G_gaussian[:,ii+1],
        line = dict(color = "firebrick"),
        name = 'Gaussian',
        hovertemplate = 'Gaussian, G(\u0394,T<sub>2r</sub>) = %{y}<br>Frequency offset = %{x} Hz<extra></extra>') for ii in range(len(t_2r))]

lineshape1[4]['visible'] = True

lineshape2 = [dict(
        visible = False,
        x = G_lor[:,0],
        y = G_lor[:,ii+1],
        line = dict(color = "royalblue"),
        name = 'Lorentzian',
        hovertemplate = 'Lorentzian, G(\u0394,T<sub>2r</sub>) = %{y}<br>Frequency offset = %{x} Hz<extra></extra>') for ii in range(len(t_2r))]

lineshape2[4]['visible'] = True

lineshape3 = [dict(
        visible = False,
        x = G_superlor[:,0],
        y = G_superlor[:,ii+1],
        line = dict(color = "orange"),
        name = 'Super Lorentzian',
        hovertemplate = 'Super Lorentzian, G(\u0394,T<sub>2r</sub>) = %{y}<br>Frequency offset = %{x} Hz<extra></extra>') for ii in range(len(t_2r))]

lineshape3[4]['visible'] = True

data = lineshape1 + lineshape2 + lineshape3

steps = []
for i in range(len(t_2r)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(lineshape1)],
        label = str(round(t_2r[i], 7))
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.0,
    active = 3,
    currentvalue = {"prefix": "T<sub>2r</sub>: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    plot_bgcolor='rgba(0,0,0,0)',
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=120,
        r=80,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.2,
            showarrow=False,
            text='Frequency offset \u0394 (Hz)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.2,
            y=0.5,
            showarrow=False,
            text='Absorption lineshape G(\u0394,T<sub>2r</sub>)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[-8000, 8000],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=True,
        type="log",
        dtick=1,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.7,
        y=0.85,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<div class=blog_body>
<p style="text-align:justify;">
In a standard qMT experiment, multiple measurements are required where the off-resonance radiofrequency pulse angle (MT angle) and offset frequency are changed for each measurement, and at least one measurement is acquired in the absence of an MT pulse. The acquired signal plotted as a function of the the offset frequency is known as the “Z-spectrum”, and Figure 3 shows the Z-spectrum simulated using different absorption lineshapes. The off-resonance MT pulse is modelled either as a continuous wave or as a pulsed irradiation scheme. In a continuous wave irradiation mode, long off-resonance pulses of constant saturation and fixed frequency offset are applied to generate well-defined MT states in the two pools (Henkelman et al. 2001). For more practical implementations, pulsed off-resonance irradiations are used with a spoiled gradient echo sequence (SPGR) to generate MT-weighted images at different saturation rates and frequency offsets (Sled and Pike 2000). In the qMT-SPGR sequence shown in Figure 1, the MT preparation pulse is applied, followed by a spoiler gradient to eliminate any residual transverse magnetization produced by the long MT pulses, and to destroy the MR signal from previous measurements. A signal equation for this type of pulsed MT experiment can be derived by approximating the pulse sequence to a series of stages described by a continuous wave off-resonance irradiation period, followed by free precession, and instant saturation of the free-water pool (Cabana et al. 2015; Sled and Pike 2000; Sled and Pike 2001). This pulse sequence decomposition allows for analytically solving the Bloch-McConnell equations at a steady-state, but it is also possible to numerically solve the Bloch equations to perform simulations that provide a more realistic approximation when the system has not been driven to a steady-state. A comparison between the Bloch simulations and the analytical solution is shown in Figure 4 for different numbers of preparation MT pulses. As can be seen, different frequency offsets and MT flip angles require different numbers of repetition times to reach the steady-state, which is of paramount importance since the analytical solution used for parameter estimation is derived assuming the steady-state. In terms of data acquisition optimization, the k-space periphery is sampled during the preparation pulses whereas the center of k-space, that contains the overall image contrast, is acquired once the steady-state is achieved.
</p>
</div>

<div class="alert alert-block alert-info">
<b>Optional code</b><br>
    
Code to generate the Z-spectrum fitted using different absorption lineshapes. <br>
    
<code>dataSim_lineshape, dataRaw_lineshape = octave.sim_lineshape(nout=2)</code>
</div>

In [ ]:
#dataSim_lineshape, dataRaw_lineshape = octave.sim_lineshape(nout=2)

In [ ]:
# Simulations have been performed and the results have been saved in the folder results.
dataSim_lineshape_mat = scipy.io.loadmat('../results/dataSim_lineshape.mat')
dataRaw_lineshape_mat = scipy.io.loadmat('../results/dataRaw_lineshape.mat')
dataSim_lineshape = np.array(dataSim_lineshape_mat["dataSim_lineshape"])
dataRaw_lineshape = np.array(dataRaw_lineshape_mat["dataRaw_lineshape"])

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 3. Z-spectrum simulated using different absorption lineshapes (Gaussian, Lorentzian, and super-Lorentzian).
</b>
</p>
</div>

In [ ]:
init_notebook_mode(connected=True)

absorption_lineshape = ["Super-Lorentzian", "Lorentzian", "Gaussian"]

fig = go.Figure()

#Add traces (three traces per fitting model)
for ii in range(len(absorption_lineshape)):
    if ii==0:
        vis = True
    else:
        vis = False
        
    fig.add_trace(go.Scatter(x=dataSim_lineshape[:,0,0], y=dataSim_lineshape[:,1,ii],
                             name="Fitted curve (angle = 142)", mode='lines', line=dict(color="firebrick"), visible = vis,
                             hovertemplate="Fitted curve (angle = 142)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>"))
    
    fig.add_trace(go.Scatter(x=dataSim_lineshape[:,0,0], y=dataSim_lineshape[:,2,ii],
                             name="Fitted curve (angle = 426)", mode='lines', line=dict(color="royalblue"), visible = vis,
                             hovertemplate="Fitted curve (angle = 426)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>"))
    
    fig.add_trace(go.Scatter(x=dataRaw_lineshape[:,0,0], y=dataRaw_lineshape[:,1,ii],
                             name="Raw data", mode='markers', line=dict(color="darkslategray"), visible = vis,
                             hovertemplate="Raw data<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>"))

buttons = []
for i, label in enumerate(absorption_lineshape):
    visibility = [False] * 9
    for j in range(3):
        visibility[3*i+j] = True
    button = dict(
                label =  label,
                method = 'update',
                args = [{'visible': visibility}])
    buttons.append(button)
        
updatemenus = list([
    dict(active=0,
         x=0.98,
         y=1.1,
         buttons=buttons
    )
])

fig['layout']['updatemenus'] = updatemenus

fig.update_layout(height=450, width=580, plot_bgcolor='rgba(0,0,0,0)',
                 margin=go.layout.Margin(
                     l=120,
                     r=80,
                     b=80,
                     t=40,
                     )
                 )
fig.update_layout(legend=dict(
        x=0.55,
        y=0.1,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2),
    annotations=[
        dict(
            x=0.35,
            y=1.1,
            showarrow=False,
            text='Absorption lineshape: ',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5004254919715793,
            y=-0.2,
            showarrow=False,
            text='Frequency offset \u0394 (Hz)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.2,
            y=0.5,
            showarrow=False,
            text='Magnetization |M<sub>z</sub>|',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ])

fig.update_xaxes(type="log", range=[2,5], showline=True, linewidth=2, linecolor='black', dtick=1, tickvals=[100,1000,10000,100000], ticktext=["10<sup>2</sup>","10<sup>3</sup>","10<sup>4</sup>","10<sup>5</sup>"])
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

iplot(fig, filename='dropdown', config=config)

<div class="alert alert-block alert-info">
<b>Optional code</b><br>
    
Code to run the Bloch simulations <br>
    
<code>numPulses = np.append(np.arange(10,100,10),np.arange(100,700,100))
numPulses = np.append(np.arange(1,6,1),numPulses)
dataSim_SNR, dataRaw_SNR = octave.BlochSim(numPulses,nout=2)</code>
</div>

In [ ]:
#numPulses = np.arange(1,6,1)
#dataSim_SNR, dataRaw_SNR = octave.BlochSim(numPulses,nout=2)

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 4. Z-spectrum simulated using Bloch simulations (dashed lines) for a number of MT pulses ranging from 1 to 600. Bloch simulations are compared with the Z-spectrum obtained from the analytical solution (solid lines).
</b>
</p>
</div>

In [ ]:
numPulses = np.append(np.arange(10,100,10),np.arange(100,700,100))
numPulses = np.append(np.arange(1,6,1),numPulses)

dataSim_Pulses_mat = scipy.io.loadmat('../results/blochSim/dataSim_Pulses.mat')
datablochSim_Pulses_mat = scipy.io.loadmat('../results/blochSim/datablochSim_Pulses.mat')
dataRaw_Pulses_mat = scipy.io.loadmat('../results/blochSim/dataRaw_Pulses.mat')
datablochSimResetMz_Pulses_mat = scipy.io.loadmat('../results/blochSim/datablochSimResetMz_Pulses.mat')
dataRawResetMz_Pulses_mat = scipy.io.loadmat('../results/blochSim/dataRawResetMz_Pulses.mat')

dataSimAnalytical_Pulses = np.array(dataSim_Pulses_mat["dataSim_Pulses"])
dataBlochSim_Pulses = np.array(datablochSim_Pulses_mat["datablochSim_Pulses"])
dataRaw_Pulses = np.array(dataRaw_Pulses_mat["dataRaw_Pulses"])
dataBlochSimResetMz_Pulses = np.array(datablochSimResetMz_Pulses_mat["datablochSimResetMz_Pulses"])
dataRawResetMz_Pulses = np.array(dataRawResetMz_Pulses_mat["dataRawResetMz_Pulses"])

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

dataSimAnalyticalPulses1 = [dict(
        visible = False,
        x = dataSimAnalytical_Pulses[:,0,0],
        y = dataSimAnalytical_Pulses[:,1,ii],
        line = dict(color = "firebrick"),
        name = 'Analytical Solution (angle = 142)',
        hovertemplate = 'Analytical Solution (angle = 142)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataSimAnalyticalPulses1[4]['visible'] = True

dataSimAnalyticalPulses2 = [dict(
        visible = False,
        x = dataSimAnalytical_Pulses[:,0,0],
        y = dataSimAnalytical_Pulses[:,2,ii],
        line = dict(color = "royalblue"),
        name = 'Analytical Solution (angle = 426)',
        hovertemplate = 'Analytical Solution (angle = 426)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataSimAnalyticalPulses2[4]['visible'] = True

dataBlochSimPulses1 = [dict(
        visible = False,
        x = dataBlochSim_Pulses[:,0,0],
        y = dataBlochSim_Pulses[:,1,ii],
        line = dict(
            color = "firebrick",
            dash = 'dash'),
        name = 'Bloch Simulation (angle = 142)',
        text = 'Bloch Simulation (angle = 142)',
        hovertemplate = 'Bloch Simulation (angle = 142)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataBlochSimPulses1[4]['visible'] = True

dataBlochSimPulses2 = [dict(
        visible = False,
        x = dataBlochSim_Pulses[:,0,0],
        y = dataBlochSim_Pulses[:,2,ii],
        line = dict(
            color = "royalblue",
            dash = 'dash'),
        name = 'Bloch Simulation (angle = 426)',
        hovertemplate = 'Bloch Simulation (angle = 426)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataBlochSimPulses2[4]['visible'] = True

dataRawPulses = [dict(
        visible = False,
        mode = 'markers',
        marker = dict(color = "darkslategray"),
        x = dataRaw_Pulses[:,0,0],
        y = dataRaw_Pulses[:,1,ii],
        name = 'Raw data',
        hovertemplate = 'Raw data<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataRawPulses[4]['visible'] = True

data = dataSimAnalyticalPulses1 + dataSimAnalyticalPulses2 + dataBlochSimPulses1 + dataBlochSimPulses2 + dataRawPulses

steps = []
for i in range(len(numPulses)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(dataSimAnalyticalPulses1)],
        label = str(numPulses[i])
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.0,
    active = 7,
    currentvalue = {"prefix": "# of Pulses: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    plot_bgcolor='rgba(0,0,0,0)',
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.2,
            showarrow=False,
            text='Frequency offset \u0394 (Hz)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.14,
            y=0.5,
            showarrow=False,
            text='Magnetization |M<sub>z</sub>|',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        type="log",
        range=[2, 5],
        dtick=1,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1.1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.05,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<center> <h2 style="font-family:timesnewroman;font-size:30px">Data Fitting</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
In qMT imaging, the biophysical model relates the parameters observed in the two-pool tissue model to physical quantities such as the fractional size of the pools, relaxation times and magnetization exchange rates of the free and restricted pool (Sled and Pike 2001; Sled 2018). However, qMT experiments usually consist of long acquisition imaging protocols accompanied by complex data fitting. To this end, some software solutions have been proposed (Karakuzu et al. 2020; Tabelow et al. 2019; Wood 2018). qMRLab is an open-source project for quantitative MR analysis that is an extension of qMTLab, a software for data simulation and analysis of three MT models: qMT-SPGR, qMT-bSSFP and qMT-SIRFSE. In addition to the quantitative MT methods, qMRLab also contains semi-quantitative MT models including the magnetization transfer ratio (MTR) and magnetization transfer saturation (MTsat).
</p>
    
<p style="text-align:justify;">
The qMT-SPGR method in qMRLab contains three fitting models: Sled and Pike, Ramani, and Yarnykh and Yuan (Sled and Pike 2001; Ramani et al. 2002; Yarnykh 2002). For the Sled and Pike model, the saturation fraction effect of the MT pulse on the free pool is pre-computed to accelerate the processing times. The MT effect of the pulse is approximated as an instantaneous fractional saturation of the longitudinal magnetization of the free pool, assuming the absence of chemical exchange processes (Pike 1996; Sled and Pike 2001). To fit the model, additional parameters related to the pulse sequence are required, namely timing parameters, the absorption lineshape, and the characteristics of the MT pulse, such as the shape and the bandwidth or the time-bandwidth product. In Figure 5, the qMT-SPGR method is used to show a single voxel curve simulation for the same MT data fitted by three different models. The fitted parameters were the pool size ratio F, the magnetization transfer rate from the restricted to the free-water pool (k<sub>r</sub>), and the transverse relaxation time of the free-water (T<sub>2,f</sub>) and restricted (T<sub>2,r</sub>) pool.
</p>
</div>

<div class="alert alert-block alert-info">
<b>Optional code</b><br>
    
Code to generate the data fitted using different models. <br>
    
<code>dataSim_model, dataRaw_model = octave.sim_modelFit(nout=2)</code>
</div>

In [ ]:
#dataSim_model, dataRaw_model = octave.sim_modelFit(nout=2)

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 5. Sled and Pike, Ramani, and Yarnykh and Yuan models to fit the MT data from a qMT-SPGR experiment.
</b>
</p>
</div>

In [ ]:
# Simulations have been performed and the results have been saved in the folder results.
dataSim_model_mat = scipy.io.loadmat('../results/dataSim_model.mat')
dataRaw_model_mat = scipy.io.loadmat('../results/dataRaw_model.mat')
dataSim_model = np.array(dataSim_model_mat["dataSim_modelFit"])
dataRaw_model = np.array(dataRaw_model_mat["dataRaw_modelFit"])

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

fitModel = ["Sled Pike RP", "Sled Pike CW", "Yarnykh", "Ramani"]

fig = go.Figure()

#Add traces (three traces per fitting model)
for ii in range(len(fitModel)):
    if ii==0:
        vis = True
    else:
        vis = False
    fig.add_trace(go.Scatter(x=dataSim_model[:,0,0], y=dataSim_model[:,1,ii],
                             name="Fitted curve (angle = 142)", mode='lines', line=dict(color="firebrick"), visible = vis,
                             hovertemplate="Fitted curve (angle = 142)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>"))
    
    fig.add_trace(go.Scatter(x=dataSim_model[:,0,0], y=dataSim_model[:,2,ii],
                             name="Fitted curve (angle = 426)", mode='lines', line=dict(color="royalblue"), visible = vis,
                             hovertemplate="Fitted curve (angle = 426)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>"))
    
    fig.add_trace(go.Scatter(x=dataRaw_model[:,0,0], y=dataRaw_model[:,1,ii],
                             name="Raw data", mode='markers', line=dict(color="darkslategray"), visible = vis,
                             hovertemplate="Raw data<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>"))


buttons = []
for i, label in enumerate(fitModel):
    visibility = [False] * 12
    for j in range(3):
        visibility[3*i+j] = True
    button = dict(
                label =  label,
                method = 'update',
                args = [{'visible': visibility}])
    buttons.append(button)
        
updatemenus = list([
    dict(active=0,
         x=0.88,
         y=1.1,
         buttons=buttons
    )
])

fig['layout']['updatemenus'] = updatemenus

fig.update_layout(height=450, width=580, plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(legend=dict(
        x=0.55,
        y=0.1,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2),
    annotations=[
        dict(
            x=0.35,
            y=1.1,
            showarrow=False,
            text='Fitting method: ',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5004254919715793,
            y=-0.2,
            showarrow=False,
            text='Frequency offset \u0394 (Hz)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.14,
            y=0.5,
            showarrow=False,
            text='Magnetization |M<sub>z</sub>|',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ])

fig.update_xaxes(type="log", range=[2,5], showline=True, linewidth=2, linecolor='black', dtick=1, tickvals=[100,1000,10000,100000], ticktext=["10<sup>2</sup>","10<sup>3</sup>","10<sup>4</sup>","10<sup>5</sup>"])
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

iplot(fig, filename='dropdown')

<div class=blog_body>
<p style="text-align:justify;">
In addition to acquiring the MT data, three more quantitative measurements are typically required for model correction purposes. The magnetic field B<sub>0</sub> inhomogeneity affects the actual off-resonance frequency experienced by the tissue at each voxel. In MT, B<sub>0</sub> maps are computed to correct for off-resonance frequency values of the MT pulse in the presence of magnetic field non-uniformity. Radiofrequency field B<sub>1</sub> inhomogeneity is another source of inaccuracies that depend on the operating frequency of the scanner, the pulse sequence, and the shape and electrical properties of the sample (Sled and Pike 1998). Therefore, B<sub>1</sub> maps are typically needed to correct the radiofrequency amplitude variations that affect the nominal values of the MT pulse flip angle and the excitation flip angle (Boudreau et al. 2018c). Longitudinal relaxation T<sub>1</sub> values vary naturally in biological tissue, but the choice of the T<sub>1</sub> mapping method, has also been shown to influence the variability of T<sub>1</sub> measurements (Stikov et al. 2015). In the context of a qMT experiment, T<sub>1</sub> maps are acquired with an independent measurement of the apparent relaxation time T<sub>1</sub> (T<sub>1</sub><sup>meas</sup>). The T<sub>1</sub> map is related to the relaxation rate of the free pool (R<sub>1,f</sub>) as described by equation 6, expressed in terms of $F$, k<sub>f</sub>, R<sub>1</sub><sup>meas</sup> and R<sub>1,r</sub>, where the relaxation rate of the restricted pool is arbitrarily set to 1 s<sup>-1</sup> because it is insensitive to this kind of MT experiments (Sled and Pike 2001). Multiple qMRI maps with a range of B<sub>0</sub> and B<sub>1</sub> inaccuracies, as well as T<sub>1</sub> maps with a variety of relaxation times, have been simulated to show the effect of the quality of these input maps on the qMT fitted parameters as shown in Figure 6.
</p>
    
<p style="text-align:justify;">
<center><img src="equation6.png" style="width:auto;height:50px;margin-bottom: 30px;margin-top: 30px;"></center>
</p>
</div>

<div class="alert alert-block alert-info">
<b>Optional code</b><br>
    
Code to fit MT data simulating input maps of different quality.<br>
    
<code>B0map = np.arange(-100,300,50)
B1map = np.arange(0.7,1.4,0.1)
R1map = np.append(np.arange(0.20,0.55,0.05),np.arange(0.6,1.6,0.1))
b0FittedData,b0NormFittedData,b1FittedData,b1NormFittedData,r1FittedData,r1NormFittedData = octave.fit_SyntheticData(B0map,B1map,R1map,nout=6)</code>
</div>

In [ ]:
#B0map = np.arange(-100,300,50)
#B1map = np.arange(0.7,1.4,0.1)
#R1map = np.append(np.arange(0.20,0.55,0.05),np.arange(0.6,1.6,0.1))
#T1map = 1/R1map
#b0FittedData,b0NormFittedData,b1FittedData,b1NormFittedData,t1FittedData,t1NormFittedData = octave.fit_SyntheticData(B0map,B1map,T1map,nout=6)

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 6. Errors (%) in fitted parameters when input maps of different quality are used. A B<sub>1</sub> map of 0.9 means that the input has a 10% lower value than expected. The fitted parameters include the pool size ratio, F, the magnetization exchange rate, k<sub>f</sub>, the free pool T<sub>2,f</sub>, and the restricted pool T<sub>2,r</sub>. The errors were simulated for B<sub>0</sub>, B<sub>1</sub> and T<sub>1</sub> maps of different quality.
</b>
</p>
</div>

In [ ]:
B0map = np.arange(-100,300,50)
B1map = np.arange(0.7,1.4,0.1)
R1map = np.append(np.arange(0.20,0.55,0.05),np.arange(0.6,1.6,0.1))
T1map = 1/R1map

#Percentage error

b1NormFittedData_mat = scipy.io.loadmat('../results/fitSyntheticData/b1NormFittedResults.mat')
b1NormFittedData = np.array(b1NormFittedData_mat["b1NormFittedResults"])

b0NormFittedData_mat = scipy.io.loadmat('../results/fitSyntheticData/b0NormFittedResults.mat')
b0NormFittedData = np.array(b0NormFittedData_mat["b0NormFittedResults"])

t1NormFittedData_mat = scipy.io.loadmat('../results/fitSyntheticData/t1NormFittedResults.mat')
t1NormFittedData = np.array(t1NormFittedData_mat["t1NormFittedResults"])

In [ ]:
init_notebook_mode(connected=True)

labels = ["B1 map", "B0 map", "T1 map"]
FittedParams = ["F", "kf (s<sup>-1</sup>)", "T2,f (s)", "T2,r"]

fig = make_subplots(rows=2, cols=2)

#F
trace1_b1 = go.Scatter(x=b1NormFittedData[:,0], y=b1NormFittedData[:,1],
                       name="F", mode='lines', line=dict(color="royalblue"), visible=True,
                       hovertemplate="Error (%) = %{y}<br>B1 value = %{x} <extra></extra>")
trace1_b0 = go.Scatter(x=b0NormFittedData[:,0], y=b0NormFittedData[:,1],
                       name="F", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>B0 value = %{x} Hz<extra></extra>")
trace1_t1 = go.Scatter(x=t1NormFittedData[:,0], y=t1NormFittedData[:,1],
                       name="F", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>T1 value = %{x} s<extra></extra>")

fig.append_trace(trace1_b1, 1, 1)
fig.append_trace(trace1_b0, 1, 1)
fig.append_trace(trace1_t1, 1, 1)

#kf
trace2_b1 = go.Scatter(x=b1NormFittedData[:,0], y=b1NormFittedData[:,2],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=True,
                       hovertemplate="Error (%) = %{y}<br>B1 value = %{x} <extra></extra>")
trace2_b0 = go.Scatter(x=b0NormFittedData[:,0], y=b0NormFittedData[:,2],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>B0 value = %{x} Hz<extra></extra>")
trace2_t1 = go.Scatter(x=t1NormFittedData[:,0], y=t1NormFittedData[:,2],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>T1 value = %{x} s<extra></extra>")

fig.append_trace(trace2_b1, 1, 2)
fig.append_trace(trace2_b0, 1, 2)
fig.append_trace(trace2_t1, 1, 2)

#T2f
trace3_b1 = go.Scatter(x=b1NormFittedData[:,0], y=b1NormFittedData[:,3],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=True,
                       hovertemplate="Error (%) = %{y}<br>B1 value = %{x} <extra></extra>")
trace3_b0 = go.Scatter(x=b0NormFittedData[:,0], y=b0NormFittedData[:,3],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>B0 value = %{x} Hz<extra></extra>")
trace3_t1 = go.Scatter(x=t1NormFittedData[:,0], y=t1NormFittedData[:,3],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>T1 value = %{x} s<extra></extra>")

fig.append_trace(trace3_b1, 2, 1)
fig.append_trace(trace3_b0, 2, 1)
fig.append_trace(trace3_t1, 2, 1)

#T2r
trace4_b1 = go.Scatter(x=b1NormFittedData[:,0], y=b1NormFittedData[:,4],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=True,
                       hovertemplate="Error (%) = %{y}<br>B1 value = %{x} <extra></extra>")
trace4_b0 = go.Scatter(x=b0NormFittedData[:,0], y=b0NormFittedData[:,4],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>B0 value = %{x} Hz<extra></extra>")
trace4_t1 = go.Scatter(x=t1NormFittedData[:,0], y=t1NormFittedData[:,4],
                       name="kf", mode='lines', line=dict(color="royalblue"), visible=False,
                       hovertemplate="Error (%) = %{y}<br>T1 value = %{x} s<extra></extra>")

fig.append_trace(trace4_b1, 2, 2)
fig.append_trace(trace4_b0, 2, 2)
fig.append_trace(trace4_t1, 2, 2)

buttons = []
for i, label in enumerate(labels):
    visibility = [i==j for j in range(len(labels))]
    button = dict(
                label =  label,
                method = 'update',
                args = [{'visible': visibility}])
    buttons.append(button)
        
updatemenus = list([
    dict(active=0,
         x=0.65,
         y=1.1,
         buttons=buttons
    )
])

fig['layout']['showlegend'] = False
fig['layout']['updatemenus'] = updatemenus

fig.update_xaxes(title_text='Input map', title_font=dict(family='Times New Roman', size=18),
                 row=1, col=1, showline=True, linewidth=2, linecolor='black')
fig.update_xaxes(title_text='Input map', title_font=dict(family='Times New Roman', size=18),
                 row=1, col=2, showline=True, linewidth=2, linecolor='black')
fig.update_xaxes(title_text='Input map', title_font=dict(family='Times New Roman', size=18),
                 row=2, col=1, showline=True, linewidth=2, linecolor='black')
fig.update_xaxes(title_text='Input map', title_font=dict(family='Times New Roman', size=18),
                 row=2, col=2, showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(range=[-100,100], row=1, col=1, showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(range=[-100,100], row=1, col=2, showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(range=[-100,100], row=2, col=1, showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(range=[-100,100], row=2, col=2, showline=True, linewidth=2, linecolor='black')

fig.update_layout(height=500, width=600, plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(annotations=[
    dict(
        x=0.20,
        y=1.1,
        showarrow=False,
        text='Input map: ',
        font=dict(
            family='Times New Roman',
            size=22
            ),
        xref='paper',
        yref='paper'
        ),
    dict(
        x=-0.1,
        y=0.90,
        showarrow=False,
        text='% Error in ' + FittedParams[0],
        font=dict(
            family='Times New Roman',
            size=18
            ),
        textangle=-90,
        xref='paper',
        yref='paper'
        ),
    dict(
        x=0.48,
        y=0.98,
        showarrow=False,
        text='% Error in ' + FittedParams[1],
        font=dict(
            family='Times New Roman',
            size=18
            ),
        textangle=-90,
        xref='paper',
        yref='paper'
        ),
    dict(
        x=-0.1,
        y=0.03,
        showarrow=False,
        text='% Error in ' + FittedParams[2],
        font=dict(
            family='Times New Roman',
            size=18
            ),
        textangle=-90,
        xref='paper',
        yref='paper'
        ),
    dict(
        x=0.48,
        y=0.07,
        showarrow=False,
        text='% Error in ' + FittedParams[3],
        font=dict(
            family='Times New Roman',
            size=18
            ),
        textangle=-90,
        xref='paper',
        yref='paper'
        )
])
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                 margin=go.layout.Margin(
                     l=120,
                     r=40,
                     b=60,
                     t=50,
                     )
                 )
fig.update_layout()

iplot(fig, filename='basic-line', config=config)

<div class=blog_body>
<p style="text-align:justify;">
As described above in Figure 6, inaccurate MT pulse flip angles and excitation flip angles affect the fitted MT parameters, and there is an additional error source related to the T<sub>1</sub> mapping measurement. As Boudreau et al. (2018c) have shown, using specific acquisitions protocols, T<sub>1</sub> values can be affected by B<sub>1</sub> field non-uniformities, such as the variable flip angle method, while the inversion recovery method is insensitive to these field inhomogeneities (Stikov et al. 2015).
</p>
    
<p style="text-align:justify;">
Figure 7 displays an example human dataset with the input qMRI maps used to fit the qMT parameters F, k<sub>f</sub>, T<sub>2,f</sub>, T<sub>2,r</sub>.
</p>
</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 7. Example magnetization transfer spoiled gradient dataset showing qMRI maps used to fit the MT data (top), and the fitted parameters F, k<sub>f</sub>, T<sub>2,f</sub>, T<sub>2,r</sub> (bottom).
</b>
</p>
</div>

In [ ]:
b0map_mat = scipy.io.loadmat('../results/realData/b0mapMAT.mat')
b0map = np.array(b0map_mat["b0mapMAT"])
b1map_mat = scipy.io.loadmat('../results/realData/b1mapMAT.mat')
b1map = np.array(b1map_mat["b1mapMAT"])
r1map_mat = scipy.io.loadmat('../results/realData/r1mapMAT.mat')
r1map = np.array(r1map_mat["r1mapMAT"])
mtdata_mat = scipy.io.loadmat('../results/realData/mtdataMAT.mat')
mtdata = np.array(mtdata_mat["mtdataMAT"])
mask_mat = scipy.io.loadmat('../results/realData/maskMAT.mat')
mask = np.array(mask_mat["maskMAT"])

Fmap_mat = scipy.io.loadmat('../results/realData/FmapMAT.mat')
Fmap = np.array(Fmap_mat["FmapMAT"])
kfmap_mat = scipy.io.loadmat('../results/realData/kfmapMAT.mat')
kfmap = np.array(kfmap_mat["kfmapMAT"])
T2fmap_mat = scipy.io.loadmat('../results/realData/T2fmapMAT.mat')
T2fmap = np.array(T2fmap_mat["T2f"])
T2rmap_mat = scipy.io.loadmat('../results/realData/T2rmapMAT.mat')
T2rmap = np.array(T2rmap_mat["T2r"])

In [ ]:
from plotly import tools

xAxis = np.arange(0,88)
yAxis = np.arange(0,128)

b0map = np.multiply(b0map,mask)
b1map = np.multiply(b1map,mask)
r1map = np.multiply(r1map,mask)
mtdata_angle = np.multiply(mtdata[:,:,:,0].reshape((88,128)),mask)
Fmap = np.multiply(Fmap,mask)
kfmap = np.multiply(kfmap,mask)
T2fmap = np.multiply(T2fmap,mask)
T2rmap = np.multiply(T2rmap,mask)

trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(b0map,3),
                   zmin=-30,
                   zmax=30,
                   colorscale='Portland',
                   colorbar=dict(title = "B<sub>0</sub> (Hz)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=True,
                   name = 'B<sub>0</sub> (Hz)')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(b1map,3),
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='Portland',
                   colorbar=dict(title = "B<sub>1</sub>", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'B<sub>1</sub> values')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(r1map,3),
                   zmin=0.5,
                   zmax=1.2,
                   colorscale='Portland',
                   colorbar=dict(title = "R<sub>1</sub> (s<sup>-1</sup>)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'R<sub>1</sub> (s<sup>-1</sup>)')
trace4 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(mtdata_angle,3),
                   zmin=0.5,
                   zmax=1,
                   colorscale='Portland',
                   colorbar=dict(title = "MT", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'MT (angle 142)')
trace5 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(Fmap,3),
                   zmin=0.05,
                   zmax=0.3,
                   colorscale='Portland',
                   colorbar=dict(title = "F", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=True,
                   name = 'F')
trace6 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(kfmap,3),
                   colorscale='Portland',
                   colorbar=dict(title = "k<sub>f</sub> (s<sup>-1</sup>)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'k<sub>f</sub> (s<sup>-1</sup>)')
trace7 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(T2fmap,3),
                   zmin=0,
                   zmax=0.05,
                   colorscale='Portland',
                   colorbar=dict(title = "T<sub>2f</sub> (ms)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15)),
                   visible=False,
                   name = 'T<sub>2f</sub> (ms)')
trace8 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(T2rmap,3),
                   zmin=5e-6,
                   zmax=25e-6,
                   colorscale='Portland',
                   colorbar=dict(title = "T<sub>2r</sub> (s)", thickness=35, ticklen=7,
                           tickfont=dict(family='Times New Roman', size=15)),
                   visible=False,
                   name = 'T<sub>2r</sub> (μs)')

data1=[trace1, trace2, trace3, trace4]
data2=[trace5, trace6, trace7, trace8]

updatemenus = list([
    dict(active=0,
         x = 0.2,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = 'B0 map',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         ]),
            dict(label = 'B1 map',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         ]),
            dict(label = 'R1 map',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         ]),
            dict(label = 'MT data',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         ])
        ]),
    )
])

layout1 = dict(
    width=345,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.5,
            y=1.15,
            showarrow=False,
            text='Input maps',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        )
    ],
    xaxis = dict(range = [0,85], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)

fig = dict(data=data1,layout=layout1)

iplot(fig, filename = 'basic-heatmap', config = config)

updatemenus = list([    
    dict(active=0,
         x = 0.2,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16),
         buttons=list([   
            dict(label = 'F map',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         ]),
            dict(label = 'kf map',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         ]),
            dict(label = 'T2f map',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         ]),
             dict(label = 'T2r map',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         ])
        ]),
    )
])

layout2 = dict(
    width=345,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.5,
            y=1.15,
            showarrow=False,
            text='Fitted maps',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        )
    ],
    xaxis = dict(range = [0,85], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)

fig = dict(data=data2,layout=layout2)

iplot(fig, filename = 'basic-heatmap', config = config)

<center> <h2 style="font-family:timesnewroman;font-size:30px">Summary</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
In summary, the Bloch-McConnell equations, an analytical solution to the steady-state signal can be derived and fitted with one of the existing models that make different approximations to fit the qMT data for a different set of parameters. The Sled and Pike model (2001) constrains the solution space by computing complementary T<sub>1</sub> maps, whose acquisition method influences the B<sub>1</sub> sensitivity of the fitted parameters (Boudreau et al. 2018c). This fitting model can be implemented with a continuous or a rectangular wave irradiation of the restricted pool (Cabana et al. 2015). Ramani’s fitting model is an alternative approach that assumes a continuous wave irradiation scheme with the MT pulse on both free and restricted pools (Ramani et al. 2002). Another fitting model was proposed by Yarnykh (2002) where an analytical solution to describe the magnetization is found when the direct saturation of the free pool is neglected.
</p>
    
<p style="text-align:justify;">
In future blog posts, we will explore other MT methods, such as MTR (Wolff and Balaban 1989), MTsat (Helms et al. 2008) and qMT-bSSFP (Brieri and Scheffler 2007; Gloor et al. 2008). Additionally, we will also be looking at the effects of RF field inhomogeneity on the generated magnetization transfer maps.
</p>
</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Works Cited</h2> </center>

<div class=biblio_body>
<p style="text-align:justify;">
Bieri, O. & Scheffler, K., 2007. Optimized balanced steady-state free precession magnetization transfer imaging. <i>Magn. Reson. Med.</i>, 58, pp.511-518. https://doi.org/10.1002/mrm.21326
</p>
    
<p style="text-align:justify;">
Boudreau, M. (2018a, October 23). Relaxometry series: Inversion recovery. qMRLab. Retrieved April 26, 2022, from https://qmrlab.org/jekyll/2018/10/23/T1-mapping-inversion-recovery.html
</p>
   
<p style="text-align:justify;">
Boudreau, M. (2018b, December 11). Relaxometry series: Variable flip angle. qMRLab. Retrieved April 26, 2022, from https://qmrlab.org/jekyll/2018/12/11/T1-mapping-variable-flip-angle.html
</p>

<p style="text-align:justify;">
Boudreau, M., Stikov, N. & Pike, G.B, 2018c. B1-sensitivity analysis of quantitative magnetization transfer imaging. <i>Magn. Reson. Med.</i>, 79, pp.276-285. https://doi.org/10.1002/mrm.26673
</p>
    
<p style="text-align:justify;">
Boudreau, M. (2019, April 08). Relaxometry series: MP2RAGE. qMRLab. Retrieved April 26, 2022, from https://qmrlab.org/2019/04/08/T1-mapping-mp2rage.html
</p>

<p style="text-align:justify;">
Cabana, J.-F., Gu, Y., Boudreau, M., Levesque, I.R., Atchia, Y., Sled, J.G., Narayanan, S., Arnold, D.L., Pike, G.B., Cohen-Adad, J., Duval, T., Vuong, M.-T. & Stikov, N., 2015. Quantitative magnetization transfer imaging made easy with qMTLab: Software for data simulation, analysis, and visualization. <i>Concepts Magn. Reson.</i>, 44A, pp.263-277. https://doi.org/10.1002/cmr.a.21357
</p>
    
<p style="text-align:justify;">
Chen, Z., Zhang, H., Jia, Z., Zhong, J., Huang, X., Du, M., Chen, L., Kuang, W., Sweeney, J.A. & Gong, Q., 2015. Magnetization Transfer Imaging of Suicidal Patients with Major Depressive Disorder. <i>Sci. Rep.</i>, 5, 9670. https://doi.org/10.1038/srep09670
</p>
    
<p style="text-align:justify;">
Fornari, E., Maeder, P., Meuli, R., Ghika, J. & Knyazeva, M.G., 2012. Demyelination of superficial white matter in early Alzheimer's disease: a magnetization transfer imaging study. <i>Neurobiol. Aging.</i>, 33(2), pp.428.e7-19. https://doi.org/10.1016/j.neurobiolaging.2010.11.014
</p>

<p style="text-align:justify;">
Gloor, M., Scheffler, K. & Bieri, O., 2008. Quantitative magnetization transfer imaging using balanced SSFP. <i>Magn. Reson. Med.</i>, 60, pp.691-700. https://doi.org/10.1002/mrm.21705
</p>

<p style="text-align:justify;">
Helms, G., Dathe, H., Kallenberg, K. & Dechent, P., 2008. High-resolution maps of magnetization transfer with inherent correction for RF inhomogeneity and T1 relaxation obtained from 3D FLASH MRI. <i>Magn. Reson. Med.</i>, 60, pp.1396-1407. https://doi.org/10.1002/mrm.21732
</p>

<p style="text-align:justify;">
Henkelman, R.M., Huang, X., Xiang, Q.-S., Stanisz, G.J., Swanson, S.D. & Bronskill, M.J., 1993. Quantitative interpretation of magnetization transfer. <i>Magn. Reson. Med.</i>, 29, pp.759-766. https://doi.org/10.1002/mrm.1910290607
</p>
    
<p style="text-align:justify;">
Henkelman, R.M., Stanisz, G.J. & Graham, S.J., 2001. Magnetization transfer in MRI: a review. <i>NMR Biomed.</i>, 14, pp.57-64. https://doi.org/10.1002/nbm.683
</p>

<p style="text-align:justify;">
Karakuzu, A., Boudreau, M., Duval, T.,Boshkovski, T., Leppert, I.R., Cabana, J.F., Gagnon, I., Beliveau, P., Pike, G.B., Cohen-Adad, J. & Stikov, N., 2020. qMRLab: Quantitative MRI analysis, under one umbrella. <i>Journal of Open Source Software</i>, 5(53), 2343 https://doi.org/10.21105/joss.02343
</p>

<p style="text-align:justify;">
Morrison, C., Stanisz, G. & Henkelman, R.M., 1995. Modeling magnetization transfer for biological-like systems using a semi-solid pool with a super-Lorentzian lineshape and dipolar reservoir. <i>J. Magn. Reson. B.</i>, 108(2), pp.103–113. https://doi.org/10.1006/jmrb.1995.1111
</p>

<p style="text-align:justify;">
Pike, G.B., 1996. Pulsed magnetization transfer contrast in gradient echo imaging: a two-pool analytic description of signal response. <i>Magn. Reson. Med.</i>, 36(1), pp.95-103.  https://doi.org/10.1002/mrm.1910360117
</p>

<p style="text-align:justify;">
Ramani, A., Dalton, C., Miller, D.H., Tofts, P.S. & Barker, G.J., 2002. Precise estimate of fundamental in-vivo MT parameters in human brain in clinically feasible times. <i>Magn. Reson. Imaging.</i>, 20(10), pp.721-731. https://doi.org/10.1016/s0730-725x(02)00598-2
</p>
    
<p style="text-align:justify;">
Seiberlich, N., Gulani, V., Campbell, A., Sourbron, S., Ivanova Doneva, M., Calamante, F. & Hu, H.H., (Eds.), 2020. Quantitative magnetic resonance imaging (1st Ed.). Elsevier.
</p>

<p style="text-align:justify;">
Sled, J.G. & Pike, G.B., 1998. Standing-wave and RF penetration artifacts caused by elliptic geometry: an electrodynamic analysis of MRI. <i>IEEE Trans. Med. Imaging.</i>, 17(4), pp.653-662. https://doi.org/10.1109/42.730409
</p>

<p style="text-align:justify;">
Sled, J.G., & Pike, G.B., 2000. Quantitative Interpretation of Magnetization Transfer in Spoiled Gradient Echo MRI Sequences. <i>Journal of Magnetic Resonance.</i>, 145(1), pp.24-36. https://doi.org/10.1006/jmre.2000.2059
</p>

<p style="text-align:justify;">
Sled, J.G. & Pike, G.B., 2001. Quantitative imaging of magnetization transfer exchange and relaxation properties in vivo using MRI. <i>Magn. Reson. Med.</i>, 46, pp.923-931. https://doi.org/10.1002/mrm.1278
</p>
    
<p style="text-align:justify;">
Stikov, N., Boudreau, M., Levesque, I.R., Tardif, C.L., Barral, J.K. & Pike, G.B., 2015. On the accuracy of T1 mapping: Searching for common ground. <i>Magn. Reson. Med.</i>, 73, pp.514-522. https://doi.org/10.1002/mrm.25135
</p>
    
<p style="text-align:justify;">
Tabelow, K., Balteau, E., Ashburner, J., Callaghan, M.F., Draganski, B., Helms, G., Kherif, F., Leutritz, T., Lutti, A., Phillips, C., Reimer, E., Ruthotto, L., Seif, M., Weiskopf, N., Ziegler, G. & Mohammadi, S., (2019). hMRI - A toolbox for quantitative MRI in neuroscience and clinical research. <i>Neuroimage.</i>, 194, pp.191-210. https://doi.org/10.1016/j.neuroimage.2019.01.029
</p>
    
<p style="text-align:justify;">
Wolff, S.D. and Balaban, R.S., (1989), Magnetization transfer contrast (MTC) and tissue water proton relaxation in vivo. <i>Magn. Reson. Med.</i>, 10, pp.135-144. https://doi.org/10.1002/mrm.1910100113
</p>

<p style="text-align:justify;">
Wood, (2018). QUIT: QUantitative Imaging Tools. <i>Journal of Open Source Software</i>, 3(26), 656. https://doi.org/10.21105/joss.00656
</p>

<p style="text-align:justify;">
Wood, T.C. & Malik, S.J., 2020. Magnetization transfer. In N. Seiberlich, V. Gulani, A. Campbell, S. Sourbron, M. I. Doneva, F. Calamante & H. H. Hu (Eds.), Quantitative Magnetic Resonance Imaging, Volume 1. (1st ed, pp.47-64). Elsevier.
</p>

<p style="text-align:justify;">
Yarnykh, V.L., 2002. Pulsed Z-spectroscopic imaging of cross-relaxation parameters in tissues for human MRI: Theory and clinical applications. <i>Magn. Reson. Med.</i>, 47, pp.929-939. https://doi.org/10.1002/mrm.10120
</p>

<p style="text-align:justify;">
Yarnykh, V.L., 2012. Fast macromolecular proton fraction mapping from a single off-resonance magnetization transfer measurement. <i>Magn. Reson. Med.</i>, 68, pp.166-178. https://doi.org/10.1002/mrm.23224
</p>
    
<p style="text-align:justify;">
Zheng, Y., Lee, J.-C., Rudick, R. & Fisher, E., 2018. Long-Term Magnetization Transfer Ratio Evolution in Multiple Sclerosis White Matter Lesions. <i>Journal of Neuroimaging</i>, 28, pp.191-198. https://doi.org/10.1111/jon.12480
</p>

</div>

In [ ]:
# PYTHON CODE

display(HTML(
    '<style type="text/css">'
    '.output_subarea {'
        'display: block;'
        'margin-left: auto;'
        'margin-right: auto;'
    '}'
    '.blog_body {'
        'line-height: 2;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
    '}'
    '.biblio_body {'
        'line-height: 1.5;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
    '}'
    '.note_body {'
        'line-height: 1.25;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
        'color: #696969'
    '}'
    '.figure_caption {'
        'line-height: 1.5;'
        'font-family: timesnewroman;'
        'font-size: 16px;'
        'margin-left: 0px;'
        'margin-right: 0px'
    '}'
    'body {margin-left:0px !important;}'
    '</style>'
))